In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [2]:
import Tr_centrality_measure
import Graph_Entropy_measure
import Selection_Algorithm
import Constraint_Coefficient

In [3]:
PI_table1 = pd.DataFrame(Graph_Entropy_measure.PI_Rank, columns = ('nodes_PI','PI_i'))
#PI_table1.PI_i[PI_table.PI_i  < 0] = 0
PI_table = PI_table1.copy()
i = math.sqrt(2)
PI_table.PI_i = PI_table.PI_i.where(lambda x: x > 0, lambda x: x + i)

In [4]:
TC_table = pd.DataFrame(Tr_centrality_measure.TC_Rank, columns = ('nodes_TC','TC_i'))
#TC_table.nodes_TC = TC_table.nodes_TC + 1
CC_table = pd.DataFrame(Constraint_Coefficient.sorted_x, columns = ('nodes_CC','CC_i'))
#CC_table.nodes_CC= CC_table.nodes_CC + 1
Result_table = pd.concat([TC_table, PI_table, CC_table], axis=1, sort=False)

First_Five= pd.DataFrame(Result_table.loc[0:2])
First_Five.index = First_Five.index + 1
First_Five

,nodes_TC,TC_i,nodes_PI,PI_i,nodes_CC,CC_i
1,0,0.4984,0,0.087362,0,0.029722
2,46,0.4982,46,0.114714,46,0.030275
3,109,0.4981,109,0.119224,3,0.037630


In [5]:
#return the total power of node i.
def TPN(i):
    PI_ofi = 1/Graph_Entropy_measure.PI_of(i)
    TC_ofi = Tr_centrality_measure.TC_of(i)
    CC_ofi = Constraint_Coefficient.CC_of(i)
    #Weight to normalize the measures
    dn=PI_ofi+TC_ofi+CC_ofi
    n1=round(TC_ofi/dn, 2)
    n2=round(PI_ofi/dn, 2)
    n3=round(CC_ofi/dn, 2)
    #equation []
    TPN_i  = (n1*TC_ofi)+(n2*PI_ofi)+(n3*CC_ofi)       
    return TPN_i

In [6]:
#loop total power of node all i
def loop_TPN():
    i=0
    nodes = list(pd.concat([First_Five['nodes_TC'], First_Five['nodes_PI'], First_Five['nodes_CC']]).unique())
    list_TPN=[]    
    for i in nodes:          
        TPN_ = TPN(i)
        i_TPN = [i, TPN_]
        list_TPN.append(i_TPN)
    return  list_TPN

In [7]:
#Ranking the total power of node i
def TPN_Ranking( val ):
      return val [1]

TPN_Rank = loop_TPN()
TPN_Rank.sort(key=lambda elem: elem[1], reverse=True)

In [8]:
#return table containing the TPN values[TPN_table and Top_Five]
TPN_table = pd.DataFrame(TPN_Rank, columns = ('nodes_TPN','TPN_i'))
Top_Five = pd.DataFrame(TPN_table.loc[0:4])
Top_Five.index = Top_Five.index + 1
#TPN_table.nodes_TPN = TPN_table.nodes_TPN + 1
print(TPN_table)
#TPN_table.to_csv('result/TPN_[]_network.csv')
#Top_Five.to_csv('result/TPN_[]_network.csv')

   nodes_TPN     TPN_i
0          0  0.417410
1         46  0.405749
2        109  0.402381
3          3  0.400268


In [9]:
#return table containing ALL the Measures
Propose_Result_table = pd.concat([TC_table, PI_table, CC_table, TPN_table], axis=1, sort=False)
Propose_Result_table.nodes_PI = Propose_Result_table.nodes_PI + 1
Propose_Result_table.nodes_TC = Propose_Result_table.nodes_TC + 1
Propose_Result_table.nodes_CC = Propose_Result_table.nodes_CC + 1
Propose_Result_table.nodes_TPN = Propose_Result_table.nodes_TPN + 1
Top_Fivep = pd.DataFrame(Propose_Result_table.loc[0:4])

#Top_Fivep.index = Top_Fivep.index + 1
#convert column nodes_TPN to int
#Top_Five.nodes_TPN= Top_Five.nodes_TPN.astype(int)
print(Top_Fivep)
#Propose_Table.to_csv('result/Propose_Table_kc_network.csv')
#Top_Five.to_csv('result/Propose_Table_power_network.csv')

   nodes_TC    TC_i  nodes_PI      PI_i  nodes_CC      CC_i  nodes_TPN  \
0         1  0.4984         1  0.087362         1  0.029722        1.0   
1        47  0.4982        47  0.114714        47  0.030275       47.0   
2       110  0.4981       110  0.119224         4  0.037630      110.0   
3         4  0.4979         5  0.122761       110  0.039479        4.0   
4         5  0.4978         4  0.129569         2  0.039623        NaN   

      TPN_i  
0  0.417410  
1  0.405749  
2  0.402381  
3  0.400268  
4       NaN  


In [10]:
#return table containing ALL the Measures
Propose_Result_table = pd.concat([TC_table, PI_table, CC_table, TPN_table], axis=1, sort=False)
Top_FiveR = pd.DataFrame(Propose_Result_table.loc[0:2])
Top_FiveR.index = Top_FiveR.index + 1
#convert column nodes_TPN to int
Top_FiveR.nodes_TPN= Top_FiveR.nodes_TPN.astype(int)
ranking_table = pd.concat([Top_FiveR['nodes_TC'], Top_FiveR['nodes_PI'], Top_FiveR['nodes_CC'],Top_FiveR['nodes_TPN']], axis=1, sort=False)
print(ranking_table)
#Propose_Table.to_csv('result/Propose_Table_power_network.csv')
#Top_Five.to_csv('result/Propose_Table_power_network.csv')

   nodes_TC  nodes_PI  nodes_CC  nodes_TPN
1         0         0         0          0
2        46        46        46         46
3       109       109         3        109


In [11]:
# calculate the kendall's correlation between two variables# calcul 
from numpy.random import rand
from numpy.random import seed
from scipy.stats import kendalltau

# prepare data
data1 = Top_FiveR['TPN_i']
data2 = data1 + Top_FiveR['CC_i']+ Top_FiveR['PI_i']+ Top_FiveR['TC_i']
# calculate kendall's correlation
coef, p = kendalltau(data1, data2)
print('Kendall correlation coefficient: %.3f' % coef)
# interpret the significance
alpha = 0.05
if p > alpha:
	print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
	print('Samples are correlated (reject H0) p=%.3f' % p)

Kendall correlation coefficient: -1.000
Samples are uncorrelated (fail to reject H0) p=0.117


In [16]:
cent = pd.read_csv('result/usair/Top_Five_cent_usair_network.csv')
all_in = pd.concat([cent,Top_Fivep], axis=1, sort=False)

Top_Five_all_in = pd.DataFrame(all_in.loc[0:4])
ranking_table_all_in = pd.concat([Top_Five_all_in['nodes_tr'], Top_Five_all_in['nodes_bc'], Top_Five_all_in['nodes_cnc'],
                                  Top_Five_all_in['nodes_dc'], Top_Five_all_in['nodes_ec'],Top_Five_all_in['nodes_pr'],
                                  Top_Five_all_in['nodes_TC'], 
                                  Top_Five_all_in['nodes_PI'], Top_Five_all_in['nodes_CC'],Top_Five_all_in['nodes_TPN']], 
                                 axis=1, sort=False)

#ranking_table_all_in.to_csv('result/ranking_table_all_in_NS.csv')
ranking_table_all_in

,nodes_tr,nodes_bc,nodes_cnc,nodes_dc,nodes_ec,nodes_pr,nodes_TC,nodes_PI,nodes_CC,nodes_TPN
0,1.0,1.0,1.0,1.0,1.0,47.0,1.0,1.0,1.0,1.0
1,47.0,206.0,47.0,47.0,47.0,1.0,47.0,47.0,47.0,47.0
2,110.0,47.0,55.0,110.0,110.0,206.0,110.0,110.0,4.0,110.0
3,4.0,2.0,110.0,4.0,4.0,2.0,4.0,5.0,110.0,4.0
4,5.0,7.0,2.0,5.0,5.0,5.0,5.0,4.0,2.0,NaN


In [17]:
# Importing required libraries
import researchpy as rp
from scipy import stats
#The casewise method returns all information in 3 dataframes
#corr_type, corr_matrix, corr_ps=>
corr_type, corr_matrix, corr_ps = rp.corr_case(Top_Five_all_in[['TR', 'BC', 'CnC', 'DC', 'EC','pr','TC_i', 'PI_i', 'CC_i', 'TPN_i']])
#corr_type, corr_matrix, corr_ps = rp.corr_case(ranking_table_all_in[['nodes_tr', 'nodes_bc', 'nodes_cnc', 'nodes_dc', 'nodes_ec','nodes_pr','nodes_TC', 'nodes_PI', 'nodes_CC', 'nodes_TPN']])
#PAIRWISEcorrelation
corr_pair = rp.corr_pair(ranking_table_all_in[['nodes_TC', 'nodes_PI', 'nodes_CC', 'nodes_TPN']])

In [22]:
corr_matrix
#corr_matrix.to_csv('result/corr_matrix_us1.csv')

,TR,BC,CnC,DC,EC,pr,TC_i,PI_i,CC_i,TPN_i
TR,1,0.9861,0.9191,0.9598,0.9977,0.8391,0.9967,-0.9205,-0.8621,0.9441
BC,0.9861,1,0.8434,0.924,0.9738,0.8417,0.9908,-0.8431,-0.878,0.8788
CnC,0.9191,0.8434,1,0.9608,0.9359,0.799,0.9054,-0.9979,-0.7824,0.9976
DC,0.9598,0.924,0.9608,1,0.9577,0.9259,0.9664,-0.9476,-0.9223,0.9735
EC,0.9977,0.9738,0.9359,0.9577,1,0.8145,0.9894,-0.9401,-0.8344,0.9574
pr,0.8391,0.8417,0.799,0.9259,0.8145,1,0.8756,-0.7646,-0.9946,0.8199
TC_i,0.9967,0.9908,0.9054,0.9664,0.9894,0.8756,1,-0.9021,-0.8986,0.9328
PI_i,-0.9205,-0.8431,-0.9979,-0.9476,-0.9401,-0.7646,-0.9021,1,0.751,-0.9955
CC_i,-0.8621,-0.878,-0.7824,-0.9223,-0.8344,-0.9946,-0.8986,0.751,1,-0.8098
TPN_i,0.9441,0.8788,0.9976,0.9735,0.9574,0.8199,0.9328,-0.9955,-0.8098,1


In [15]:
#Graph_Entropy_measure.PI_of(23)